In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

from selenium.webdriver.chrome.options import Options 
from selenium import webdriver

import pandas as pd
import urllib.request
import requests
import string
from bs4 import BeautifulSoup
from nltk.corpus import stopwords #kick the words out such as 'the', 'a'
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
%matplotlib inline
from tqdm import tqdm

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
from nltk.corpus import stopwords #kick the words out such as 'the', 'a'

In [5]:
# Load the Loughran and McDonald dictionnary
lm = pd.read_excel('../engineering project/LoughranMcDonald_MasterDictionary_2020.xlsx')

# get the lists of positive/negative words
lmpos = list(lm[lm.Positive!=0]['Word'])
lmneg = list(lm[lm.Negative!=0]['Word'])

In [6]:
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def get_article_body(article_url):
    
    # 셀레니움을 이용한 크롤링
    
    driver = webdriver.Chrome(executable_path='../driver/chromedriver.exe') ################## 드라이버 경로는 각자 컴퓨터에 맞게!!!
    
    page = driver.get(article_url)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    html_text = soup.find('div',{'class':'caas-body'})    
    div = soup.find('div',{'class':'caas-attr-time-style'})   # 기사 내용이 div class='caas-body'에 들어있음 (하이퍼링크내용 제거는 차후 코딩...)
    date = div.text                                           # text화 시켜 하나의 내용으로 변경
    
    driver.close()
    
    stripped_text = []
    for t in html_text:
        t = strip_tags(str(t))
        stripped_text.append(t)
    
    return " ".join(stripped_text)
    

In [ ]:
cciv = pd.read_excel('NIKE.xlsx') # 엑셀파일이름과 동일하게

In [ ]:
cciv = cciv.rename(columns=cciv.iloc[0])
cciv

In [ ]:
cciv.drop(index=0,inplace=True)
cciv

In [ ]:
cciv = cciv[['Altria']] #### 위 DF 의 Facebook(열 이름) 과 같아야함
cciv

# 모든 xlsx은 아래와 같은 형식의 DF로 있어야 점수측정 가능!!!!!!!!!!!!!

In [ ]:
cciv_list = cciv['Altria'].to_list() #### 위 DF 의 Facebook(열 이름) 과 같아야함

In [ ]:
nwords = []
nneg = []
npos = []
# c_date=[]

# in this block of code, we extract informations from each article to fill up
# the lists above
# for i in range(len(cciv_list)):
for i in tqdm(cciv_list):  ########################### NONE값 0점 확인을 위해 cciv_list[4:6]로 함 ###########################
    if i == 'NONE':
        nwords.append(0)
        npos.append(0)
        nneg.append(0)
#         c_date.append('NONE')
    else:
        # let's use the helper function to provide us with the article body
        # New York Times wraps their body in the <p> HTML element. The website you use might be different
        article_body = get_article_body(i)

        # convert all words to lower case
        article_body = article_body.lower()

        # remove non-letters (i.e., $ %)
        natural_lang_data = ''.join([x for x in article_body if x in string.ascii_letters + '\'- '])

        # before processing the natural language data, we need to remove stop words.
        # stop words are words like: about, that, this, and, or, etc...
        natural_lang_data = [x for x in natural_lang_data.split() if x not in stopwords.words('english')]

        # finally, we store the following data: 
        nwords.append(len(natural_lang_data)) # number of words
        npos.append(len([i for i in natural_lang_data if i.upper() in lmpos]))  # number of positive words
        nneg.append(len([i for i in natural_lang_data if i.upper() in lmneg]))  # number of negative words
#         c_date.append(date)

In [ ]:
cciv_data = pd.DataFrame(data={'nwords':nwords, 'npos':npos, 'nneg':nneg})
cciv_data

In [ ]:
cciv_data.to_csv('Altria score.csv')